## Introduction
In this case study, we show Pacti can work with a contract-based methodology to aid word-length optimization of digital signal processing integrated circuit design.

A digital signal processing system takes digital signals as inputs and generates the output signals by applying digital filters.
The input signals are usually obtained through an analog-to-digital converter(ADC) from analog signals acquired from the environment by sensors.
The impulse response of the digital filter is a set of coefficients that multiply with the input signals. The result is summed up to get the output signals.

Implementation of a digital filter in hardware is a time-consuming and error-prone process.
To push the power, performance, and area to their limits, the floating-point operations are converted to fixed-point ones to reduce hardware costs and improve efficiency.
However, the conversion incurs challenges, including the risk of overflow and lower accuracy.
As a fixed-point number can represent smaller range than a floating-point one with the same number of bits, overflow might occur after the conversion.
The fixed-point number with fewer bits tends to have lower accuracy as all bits in the fractional parts exceeding its fractional bits have to be discarded.

The following figure shows a multiplication operation using fixed-point numbers.
![Data_Path](./figures/digital_data_path_example.png)

As a result, verifying and optimizing word length for fixed-point implementation is crucial for integrated circuit implementations.
To prevent overflow and ensure the inaccuracy is acceptable, we need to bound the error of the system given the wordlength. 
In this case study, we apply contracts to compute the bounds of error of fixed-point arithmetic implementation.

## Preliminaries

First, we introduce the fixed-point number.
#### Representation of Fixed-point Number

A word length of a fixed-point variable $x$ can denoted as a tuple $(x_n, x_p)$, where $x_n$ denotes the number of bits, and $x_p$ is number of bits for the integer parts.
For example, A fixed-point number with word length $(5,2)$ with bit value "11011" means the binary number $11.011$, equivalent to $3.375$ in decimal.

First, we import some data structure for holding the wordlength, and the APIs of Pacti.

In [1]:
from pacti.terms.polyhedra.loaders import read_contract, write_contract
from tool import *
import numpy as np

In [2]:
# Definition of value_num method in PortWordLength class
#    def value_num(self):
#        return int(self.value, base=2) * 2 ** (- (self.n - self.p))

port = PortWordLength(n=5, p=2, value="11011")
print(f"The word length (n = {port.n}, p = {port.p}) of the bits \"{port.value}\" is: ")
print(port.value_num())

The word length (n = 5, p = 2) of the bits "11011" is: 
3.375


#### Error Sources in Fixed-point Arithmetic
The error of Fixed-point arithmetic comes from the following sources:
1. Truncation Errors:
A truncation error occurs when a fixed-point number is converted from a longer representation to a shorter one.
As the number of bits is insufficient to represent the number perfectly, the least significant bits are truncated to match the word length.
Truncation errors occur in fixed-point operations when the word length of the output variable is smaller than the ideal result.
2. Quantization Error:
The quantization error occurs when an analog signal is converted into a digital one or when the coefficients are encoded into digital formats. As fixed-point number cannot represent any real number with infinite precision, the signals and coefficients have to be quantized, and errors are introduced in the process.
3. Inaccurate Source:
This error comes from the measurement in the analog signal because of noise and non-ideal effect on the sensor and could be seen as a random noise on the analog signal.

The following figure shows the three types of error in the example.
![Error_Type](./figures/type_of_error.png)

Given a input analog signal, the input signal and filter coefficients are first quantized into fixed-point numbers.
Then the arithmetic operations, based on the algorithm or filter design, are performed on these fixed-point numbers.
The results of the operations are truncated to fit into the fixed-point numbers of the output of the arithmetic operations.

During the process, differences between the fixed-point numbers representation and the ideal numbers result in errors.
These errors propagate through the operations and thus affect the accuracy of the output signal.

## Forming contract for capturing error of fixed-point number

Here we introduce how to form the contract for finding error bounds of the fixed-point number operation.
First, we model the truncation errors and investigate how an operation result in the truncation errors.
Then we consider the propagation of the error in different operations.
We formulate the contract by combining the truncation errors and propagation of the error.

#### Modeling of Error from Operations
Each operation can be seen as a two-staged process.
First, an ideal output with certain wordlength is generated without losing any accuracy, given the word lengths of all input ports.
Then the ideal output is truncated to the actual output port.
Therefore, the modeling of the ideal output wordlength and the truncation error is required to formulate the contract.

#### Operation Error Modeling
In the following, we detail how to model the truncation error.
We divide the truncation error into two cases according to the number of bits for integer part.

##### Case 1: Identical Number of Bits in Integer Part
As the integer part has the same number of bits, overflow does not occur in this case. The least significant bits are truncated if the resulting word length is shorter. The maximal truncation error from the word length $(x_n, x_p)$ to the word length $(x_n', x_p)$ is $2^{x_p}(2^{-x_n'}-2^{-x_n})$, when $x_n > x_n'$. If $x_n < x_n'$, there is no truncation, and thus the truncation error is exactly $0$.

The following example shows how the error is bounded:

In [3]:
def truncation_error_same_position(pi, po):
    assert(pi.p == po.p)
    if pi.n > po.n:
        return 2 ** po.p * (2 ** (-po.n) - 2 ** (-pi.n))
    else :
        return 0

def truncate(pi, po):
    # separate the bits into two parts, before binary point and after binary point
    bits_before_point = pi.value[:pi.p]
    bits_after_point = pi.value[pi.p:]
    
    # truncate or appending 0 
    if po.p >= pi.p:
        bits_before_point = ("0" * (po.p - pi.p)) + bits_before_point
    else:
        bits_before_point = bits_before_point[pi.p - po.p:]

    if po.n - po.p >= pi.n - pi.p:
        bits_after_point = bits_after_point + "0" * (po.n - po.p - pi.n + pi.p)
    else:
        bits_after_point = bits_after_point[:(po.n - po.p)]

    # combine the both parts
    ret = bits_before_point + bits_after_point
    po.set_value(value = ret)
    return po.value

In [4]:
p1 = PortWordLength(n=7, p=2, name="p1", value = "1111111")
p2 = PortWordLength(n=5, p=2, name="p2")

print("Truncation Error: ")
print(truncation_error_same_position(p1, p2))
print("Example Value to produce the error")

truncate(p1, p2)
print(f"p1 = {p1.value_num()}, bits: {p1.value}")
print(f"p2 = {p2.value_num()}, bits: {p2.value}")
print(f"p1 - p2 = ", p1.value_num() - p2.value_num())

Truncation Error: 
0.09375
Example Value to produce the error
p1 = 3.96875, bits: 1111111
p2 = 3.875, bits: 11111
p1 - p2 =  0.09375


Then let's see what happens if the integer parts have different numbers of bits.
##### Case 2. Different Number of Bits in Integer Part 
As the integer parts have different numbers of bits, overflow could occur if the resulting number cannot hold the original integer values.
If an overflow occurs, the result deviates from the ideal value, and thus we expect no overflow occurs during the computation.
This expectation will be applied as the assumption in subsequent parts when we formulate contracts.

Consider the case when converting a variable $x$ with word length $(x_n, x_p)$ to the variable $x'$ with word length $(x_n', x_p')$.
We have to ensure that the value of $x$ does not incur overflow in $x'$.
Therefore, it is reasonable to assume that we know that $x$ is bounded by $2^{x_p'}$ so that $x'$ can represent the number without overflows. 

Then, we can ignore the unnecessary most significant bits ($x_p - x_p'$), as they are always $0$s under the assumption.
As a result, $(x_n, x_p)$ can be seen as $(x_n - x_p + x_p', x_p')$. 
The two numbers now have identical numbers of bits in their integer parts, and thus we can apply case 1 to get the truncation errors under the assumption.

The following example shows how the error can be calculated.

In [5]:
def truncation_error(pi, po):
    # remove uneccesary most significant bits
    # Remider: The assumption must hold!!
    pi_adjusted = PortWordLength(n=pi.n - pi.p + po.p, p=po.p)
    # return the truncation error as in case 1
    return truncation_error_same_position(pi=pi_adjusted, po=po)

def get_assumption_value(pi, po):
    if pi.p > po.p:
        return 2 ** po.p
    else:
        return float("inf") # no additional assumption needed

def check_assumption_value(pi, po):
    assumption_value = get_assumption_value(pi, po)
    print(f"Assumption: {pi.name} < {assumption_value}")
    if not pi.value_num() < assumption_value:
        print("Assumption failed, truncation of MSB occurs")
        return False
    else:
        print(f"Assumption Satisfied ({pi.name} = {pi.value_num()} < {assumption_value})")
        return True
        


In [6]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")

assert(check_assumption_value(pi=p1, po=p2))

print("Truncation Error: ")
print(truncation_error(p1, p2))
print("Example Value to produce the error")

truncate(p1, p2)
print(f"p1 = {p1.value_num()}, bits: {p1.value}")
print(f"p2 = {p2.value_num()}, bits: {p2.value}")
print(f"p1 - p2 = ", p1.value_num() - p2.value_num())

Assumption: p1 < 4
Assumption Satisfied (p1 = 2.4375 < 4)
Truncation Error: 
0.0625
Example Value to produce the error
p1 = 2.4375, bits: 0100111
p2 = 2.375, bits: 10011
p1 - p2 =  0.0625


#### Ideal Output Wordlength
Now we know how to get the truncation errors given the word length of two numbers, but what is the word length truncated in operations?

Each operation requires a minimum ideal output word length to fully hold the resulting value without losing any information.
Assuming a binary operator, with inputs being $x$ and $y$ and output being $z$, we represent the ideal output word length using $(z_n^*, z_p^*)$.
The ideal output word length is then truncated to fit the output port.

Therefore, we need to know the ideal output word length for the operations.
Let's first examine the addition and then the multiplication.

##### Addition
The following equations show the ideal output word length for addition.

$z_n^* = \max(x_n, y_n - y_p + x_p) + \min(0, x_p - y_p) + 1$,

$z_p^* = \max(x_p, y_p) + 1$.

The rationale behind the equations is to keep every bit from both inputs and include an additional bit for the carry.

In [7]:
def compute_required_word_length_add(in1: PortWordLength, in2:PortWordLength) -> PortWordLength:
    new_n = max(in1.n, in2.n - in2.p + in1.p) - min(0, in1.p - in2.p) + 1
    new_p = max(in1.p, in2.p) + 1
    return PortWordLength(n=new_n, p=new_p)

def error_truncation_add(p1, p2, po):
    print(f"Input {p1.name}: (n={p1.n}, p={p1.p})")
    print(f"Input {p2.name}: (n={p2.n}, p={p2.p})")
    p_ideal = compute_required_word_length_add(in1=p1, in2=p2)
    print(f"Ideal Output: (n={p_ideal.n}, p={p_ideal.p})")
    print(f"Actual Output ({po.name}): (n={po.n}, p={po.p})")

    assumption_value = get_assumption_value(pi=p_ideal, po=po)
    print(f"Assumption: {p1.name} + {p2.name} < {assumption_value}")
    return truncation_error(pi=p_ideal, po=po)
    

In [8]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

err = error_truncation_add(p1=p1, p2=p2, po=p3)

print(f"Error of the addition calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=8, p=4)
Actual Output (p3): (n=5, p=3)
Assumption: p1 + p2 < 8
Error of the addition calculation: 0.1875


##### Multiplication
The following equations show the ideal output word length for multiplication.

$z_n^* = x_n + y_n$,

$z_p* = x_p, y_p$.

In [9]:
def compute_required_word_length_mult(in1: PortWordLength, in2:PortWordLength) -> PortWordLength:
    new_n = in1.n + in2.n
    new_p = in1.p + in2.p
    return PortWordLength(n=new_n, p=new_p)

def error_truncation_mult(p1, p2, po):
    print(f"Input {p1.name}: (n={p1.n}, p={p1.p})")
    print(f"Input {p2.name}: (n={p2.n}, p={p2.p})")
    p_ideal = compute_required_word_length_mult(in1=p1, in2=p2)
    print(f"Ideal Output: (n={p_ideal.n}, p={p_ideal.p})")
    print(f"Actual Output ({po.name}): (n={po.n}, p={po.p})")

    assumption_value = get_assumption_value(pi=p_ideal, po=po)
    print(f"Assumption: {p1.name} * {p2.name} < {assumption_value}")
    return truncation_error(pi=p_ideal, po=po)

In [10]:
p1 = PortWordLength(n=7, p=3, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

err = error_truncation_mult(p1=p1, p2=p2, po=p3)

print(f"Error of the multiplication calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=12, p=5)
Actual Output (p3): (n=5, p=3)
Assumption: p1 * p2 < 8
Error of the multiplication calculation: 0.2421875


#### Propagation of Error
Since fixed-point arithmetic incurs errors, the error could propagate through the previous operation to the final output.
This error can be seen as independent of truncation error, as truncation errors are applied to the actual input numbers, which already contain the propagation errors.
As a result, the error in the output variable $z$, denoted as $z_e$, can be represented as the sum of the propagation error and the truncation error in the stage: 

$z_e = z_{et} + z_{ep}$,

where $z_{ep}$ is the propagation error from inputs, and $z_{et}$ denotes the truncation error occurs in the operation.

Let's now see how the error is propagated in different operations.

##### General Operation
The propagation error is bounded by the maximum differences between the ideal value and the actual values.
Therefore, the propagation errors can be modeled as follows: 

$z_{ep} = \max_{0<=x<=x_a, 0<=y<=y_a}(f(x, y)) - f(x-x_e, y-y_e))$,

where $x_{a}$ and $y_{a}$ is the upper bound of input variables $x$ and $y$, respectively.

According to this equation, we can derive the propagation error for addition and multiplication operations.

##### Propagation of Error in Addition
The error is exactly the sum of error from both inputs.
$z_{ep} = x_{e} + y_{e}$

In [11]:
def error_propagation_add(p1, p2, po):
    return p1.e + p2.e

In [12]:
p1 = PortWordLength(n=7, p=3, e=0.5, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, e=0.2, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

e_p = error_propagation_add(p1=p1, p2=p2, po=p3)
e_t = error_truncation_add(p1=p1, p2=p2, po=p3)

err = e_p + e_t
print(f"Propagation error: {e_p}")
print(f"Truncation error: {e_t}")
print(f"Error bound of the addition calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=8, p=4)
Actual Output (p3): (n=5, p=3)
Assumption: p1 + p2 < 8
Propagation error: 0.7
Truncation error: 0.1875
Error bound of the addition calculation: 0.8875


##### Propagation of Error in Multiplication
The error is as follows:

$z_{ep} = x_{a}*y_{e} + y_{a}*x_{e} - x_{e} * y_{e}$, 



In [13]:
def get_actual_possible_value(in_port: PortWordLength):
    return (1 - (2 ** -in_port.n) )* 2 ** in_port.p

def error_propagation_mult(p1, p2, po):
    return p1.a * p2.e + p2.a * p1.e + p1.e * p2.e

In [14]:
p1 = PortWordLength(n=7, p=3, e=0.5, name="p1", value = "0100111")
p2 = PortWordLength(n=5, p=2, e=0.3, name="p2")
p3 = PortWordLength(n=5, p=3, name = "p3")

e_p = error_propagation_mult(p1=p1, p2=p2, po=p3)
e_t = error_truncation_mult(p1=p1, p2=p2, po=p3)

err = e_p + e_t
print(f"Propagation error: {e_p}")
print(f"Truncation error: {e_t}")
print(f"Error bound of the multiplication calculation: {err}")

Input p1: (n=7, p=3)
Input p2: (n=5, p=2)
Ideal Output: (n=12, p=5)
Actual Output (p3): (n=5, p=3)
Assumption: p1 * p2 < 8
Propagation error: 2.81875
Truncation error: 0.2421875
Error bound of the multiplication calculation: 3.0609375


#### Contract Formulation
we can formulate contracts for each operation by combining the modeling of error bounds from propagation and truncation.
A contract is a pair of an assumption and a guarantee $C = (A, G)$, where $C$ represents the contract, $A$ denotes its assumption, and $G$ is the guarantee.
In the following, we shall introduce the contract formulation for different operations.

##### Ports in Contract

Contracts specify system behaviors by describing the assignment of values in ports. 
For our case study, we see a variable $x$ as a combination of two ports: the upper bound value port and the upper bound error port. 
The upper bound value port $x_a$ tells the connected component the maximum value the variable can be, and the upper bound error port $x_e$   passes the maximum error to the connected component.

##### Assumption
The assumption ensures no overflow occurs during the truncation, which has been introduced in the case where truncation involves two variables with different numbers of bits in the integer parts.

##### Guarantee
The guarantee of the contract includes two parts. The first part describes the upper bound value, and the second states the upper bound error.

The following functions show how we form contract for arbitrary length of input ports and output ports.
The "_a" variables are used as the variable for the actual value going through the port.

In [15]:
def form_contract_add(in_port1, in_port2, out_port):
    ret_contract = {}
    # define input/output vars
    ret_contract["InputVars"] = [f"{in_port1.name}_a", f"{in_port1.name}_e",
                                 f"{in_port2.name}_a", f"{in_port2.name}_e"]
    ret_contract["OutputVars"] = [f"{out_port.name}_a", f"{out_port.name}_e"]
    # get assumption
    ideal_out_port = compute_required_word_length_add(in1=in_port1, in2=in_port2)
    assumption_value = get_assumption_value(pi=ideal_out_port, po=out_port)
    # write assumption in the contract
    if assumption_value != float("inf"):
        ret_contract["assumptions"] =  [{"coefficients":{f"{in_port1.name}_a":1, f"{in_port2.name}_a":1},
                                    "constant":assumption_value}]
    else:
        ret_contract["assumptions"] = []
    # get guarantee
    e_t = truncation_error(pi=ideal_out_port, po=out_port)

    # write guarantee in the contract, note the propagation is encoded in the polyhedral constraints
    ret_contract["guarantees"] =  [{"coefficients":{f"{in_port1.name}_e":-1, f"{in_port2.name}_e":-1, f"{out_port.name}_e": 1},
                                "constant":e_t},
                                {"coefficients":{f"{out_port.name}_a": 1}, "constant":out_port.a},
                                #{"coefficients":{f"{out_port.name}_a": 1}, "constant":in_port1.a + in_port2.a},
                                {"coefficients":{f"{out_port.name}_a": 1, f"{in_port1.name}_a": -1, f"{in_port2.name}_a": -1}, "constant":0}
                                ]
    return ret_contract


def form_contract_mult_const(in_port1, in_port_const, out_port):
    ret_contract = {}
    # define input/output vars
    ret_contract["InputVars"] = [f"{in_port1.name}_a", f"{in_port1.name}_e"]
    ret_contract["OutputVars"] = [f"{out_port.name}_a", f"{out_port.name}_e"]
    # get assumption
    ideal_out_port = compute_required_word_length_mult(in1=in_port1, in2=in_port_const)
    assumption_value = get_assumption_value(pi=ideal_out_port, po=out_port)
    # write assumption in the contract
    if assumption_value != float("inf"):
        print(assumption_value)
        ret_contract["assumptions"] =  [{"coefficients":{f"{in_port1.name}_a": in_port_const.a},
                                    "constant":assumption_value}]
    else:
        ret_contract["assumptions"] = []
    # get guarantee
    print(ideal_out_port.to_string())
    e_t = truncation_error(pi=ideal_out_port, po=out_port)

    # write guarantee in the contract, note the propagation is encoded in the polyhedral constraints
    ret_contract["guarantees"] =  [{"coefficients":
                                    {   f"{in_port1.name}_e":-in_port_const.a+in_port_const.e, 
                                        f"{in_port1.name}_a":-in_port_const.e,
                                        f"{out_port.name}_e": 1},
                                    "constant":e_t},
                                    {"coefficients":{f"{out_port.name}_a": 1}, "constant":out_port.a},
                                    #{"coefficients":{f"{out_port.name}_a": 1}, "constant":in_port_const.a * in_port1.a},
                                    {"coefficients":{f"{out_port.name}_a": 1, f"{in_port1.name}_a": -in_port_const.a}, "constant":0}
                                    ]
    return ret_contract

In [16]:
p1 = PortWordLength(n=7, p=3, name="p1")
p2 = PortWordLength(n=5, p=2, name="p2")
p3 = PortWordLength(n=5, p=3, name="p3")
c1 = form_contract_add(in_port1=p1, in_port2=p2, out_port=p3)
print(c1)
contract1 = read_contract(c1)
print(str(contract1))


p4 = PortWordLength(n=7, p=3, name="p4")
p5 = PortWordLength(n=5, p=2, e=0.03, value="11011", name="p5") # const
p6 = PortWordLength(n=5, p=3, name="p6")
c2 = form_contract_mult_const(in_port1=p4, in_port_const=p5, out_port=p6)
contract2 = read_contract(c2)
print(str(contract2))

{'InputVars': ['p1_a', 'p1_e', 'p2_a', 'p2_e'], 'OutputVars': ['p3_a', 'p3_e'], 'assumptions': [{'coefficients': {'p1_a': 1, 'p2_a': 1}, 'constant': 8}], 'guarantees': [{'coefficients': {'p1_e': -1, 'p2_e': -1, 'p3_e': 1}, 'constant': 0.1875}, {'coefficients': {'p3_a': 1}, 'constant': 7.75}, {'coefficients': {'p3_a': 1, 'p1_a': -1, 'p2_a': -1}, 'constant': 0}]}
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>]
OutVars:[<Var p3_a>, <Var p3_e>]
A: 1*p1_a + 1*p2_a <= 8.0
G: -1*p1_e + -1*p2_e + 1*p3_e <= 0.1875, 1*p3_a <= 7.75, -1*p1_a + -1*p2_a + 1*p3_a <= 0.0
8
Port: , (n, p) = (12, 5), e = 0, a = 31.9921875
InVars: [<Var p4_a>, <Var p4_e>]
OutVars:[<Var p6_a>, <Var p6_e>]
A: 3.375*p4_a <= 8.0
G: -0.03*p4_a + -3.345*p4_e + 1.0*p6_e <= 0.2421875, 1.0*p6_a <= 7.75, -3.375*p4_a + 1.0*p6_a <= 0.0


#### Example 1
Consider the following simple system with only two adder.

![Example 1](./figures/example1.png)

We will show that how Pacti can be applied to get the system errors.

First, we express each port using the PortWordLength class, and then form the contracts for the operations.

In [17]:
def create_example1():
    p1 = PortWordLength(n=5, p=2, name = "p1")
    p2 = PortWordLength(n=5, p=3, name = "p2")
    p3 = PortWordLength(n=5, p=3, name = "p3")
    c1 = form_contract_add(in_port1=p1, in_port2=p2, out_port=p3)

    p4 = PortWordLength(n=7, p=3, name = "p4")
    p5 = PortWordLength(n=6, p=3, name = "p5")
    c2 = form_contract_add(in_port1=p3, in_port2=p4, out_port=p5)

    contract1 = read_contract(c1)
    contract2 = read_contract(c2)
    print("Contract 1:\n" + str(contract1))
    print("Contract 2:\n" + str(contract2))
    return contract1, contract2, p1, p2, p3, p4, p5
    
contract1, contract2, p1, p2, p3, p4, p5 = create_example1()

Contract 1:
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>]
OutVars:[<Var p3_a>, <Var p3_e>]
A: 1*p1_a + 1*p2_a <= 8.0
G: -1*p1_e + -1*p2_e + 1*p3_e <= 0.125, 1*p3_a <= 7.75, -1*p1_a + -1*p2_a + 1*p3_a <= 0.0
Contract 2:
InVars: [<Var p3_a>, <Var p3_e>, <Var p4_a>, <Var p4_e>]
OutVars:[<Var p5_a>, <Var p5_e>]
A: 1*p3_a + 1*p4_a <= 8.0
G: -1*p3_e + -1*p4_e + 1*p5_e <= 0.0625, 1*p5_a <= 7.875, -1*p3_a + -1*p4_a + 1*p5_a <= 0.0


We then use the composition to get the system contract.

In [18]:
contract_sys = contract1.compose(contract2)
print("Contract Sys:\n" + str(contract_sys))

Contract Sys:
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>, <Var p4_a>, <Var p4_e>]
OutVars:[<Var p5_a>, <Var p5_e>]
A: 1*p4_a <= 0.250000000000000, 1*p1_a + 1*p2_a <= 8.0
G: -1*p1_e + -1*p2_e + -1*p4_e + 1*p5_e <= 0.187500000000000, -1*p4_a + 1*p5_a <= 7.75, -1*p1_a + -1*p2_a + -1*p4_a + 1*p5_a <= 0.0, 1*p5_a <= 7.875


The system contracts show the relation between the actual value and error bounds in input values.

If the designers know the context how the system is used, they can includes additional contracts to constrain the input:

First, we try an example that has no additional contraint on the input, meaning that it can take arbitrary value for the fixed-point number.

In [19]:
def form_contract_input(in_port):
    ret_contract = {}
    # define input/output vars
    ret_contract["InputVars"] = []
    ret_contract["OutputVars"] = [f"{in_port.name}_a", f"{in_port.name}_e"]
    # get assumption
    ret_contract["assumptions"] = []
    ret_contract["guarantees"] =  [ {"coefficients": {f"{in_port.name}_a": 1}, "constant": in_port.a},
                                    {"coefficients": {f"{in_port.name}_a": -1}, "constant": 0},
                                    {"coefficients": {f"{in_port.name}_e": 1}, "constant": in_port.e},
                                    {"coefficients": {f"{in_port.name}_e": -1}, "constant": -in_port.e}]
    return ret_contract

In [20]:
def test_example_1():
    c_p1 = form_contract_input(in_port=p1)
    c_p2 = form_contract_input(in_port=p2)
    c_p4 = form_contract_input(in_port=p4)

    contract_p1 = read_contract(c_p1)
    contract_p2 = read_contract(c_p2)
    contract_p4 = read_contract(c_p4)
    
    try: 
        contract_sys = contract1.compose(contract2)
        contract_sys = contract_p1.compose(contract_sys)
        contract_sys = contract_p2.compose(contract_sys)
        contract_sys = contract_p4.compose(contract_sys)
        print("Contract Sys:\n" + str(contract_sys))
        return contract_sys
    except ValueError as e:
        print("Composition Error")
        print(e)

contract_sys = test_example_1()

Composition Error
The guarantees 
1*p2_a <= 7.75, -1*p2_a <= 0.0, 1*p2_e <= 0.0, -1*p2_e <= 0.0
were insufficient to abduce the assumptions 
1*p4_a <= 0.250000000000000, 1*p2_a <= 4.12500000000000
by eliminating the variables 
[<Var p2_a>, <Var p2_e>, <Var p5_a>, <Var p5_e>]


The result shows that the guarantees are not sufficient to abduce the assumption.

It can be observed easily that the input value of `p2_a` = 7.75 would violate the required assumption to prevent MSB from truncated.


Then we consider the case where certain constraints are known for the inputs:

In [21]:
p1.set_value("10000")
p2.set_value("01111")
p4.set_value("0000001")
contract_sys = test_example_1()

Contract Sys:
InVars: []
OutVars:[<Var p5_a>, <Var p5_e>]
A: true
G: 1*p5_a <= 5.81250000000000, 1*p5_e <= 0.187500000000000


Under these sets of constraints on the input, the system is compatible with the environment and thus we obtain the bounds on the actual output and the error bounds to an ideal output. 

The contract bring us the benefits of performing more operations on the system.

For example, we can apply the quotient to find word length for a certain signal.

Take the system as an example again. If we want to reduce the wordlength of the intermediate signal $P_3$, we can use quotient to get the required wordlength.

![Example 1](./figures/example1.png)

To use quotient, we starts from the system contract.
The system contract describes what the system looks like without knowing the underlying components.
Let's say we can tolerate an error of the output by 0.1 given the same input constraints for the previous refined one.

The system contract should be like this:
```
A: True
G: 1*p5_e <= 0.1
```

We can invoke contract quotient to get the desired system contract:

In [22]:
def get_desired_system_contract():
    ret_contract = {}
    ret_contract["InputVars"] = []
    ret_contract["OutputVars"] = ["p5_a", "p5_e"]
    ret_contract["assumptions"] = []
    ret_contract["guarantees"] = [
                                  {"coefficients": {f"p5_e": 1}, "constant": 0.1}]
    return read_contract(ret_contract)

contract_spec = get_desired_system_contract()
print("Top Level Specification:")
print(str(contract_spec))

Top Level Specification:
InVars: []
OutVars:[<Var p5_a>, <Var p5_e>]
A: true
G: 1*p5_e <= 0.1


In [23]:
def quotient_example1():
    c_p1 = form_contract_input(in_port=p1)
    c_p2 = form_contract_input(in_port=p2)
    c_p4 = form_contract_input(in_port=p4)

    contract_p1 = read_contract(c_p1)
    contract_p2 = read_contract(c_p2)
    contract_p4 = read_contract(c_p4)

    tmp_c1 = contract_spec.quotient(contract_p1)

    tmp_c2 = tmp_c1.quotient(contract_p2)
    tmp_c3 = tmp_c2.quotient(contract_p4)
    return tmp_c3

quotient_ret = quotient_example1()
print("Quotient of the Spec:")
print(str(quotient_ret))


Quotient of the Spec:
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>, <Var p4_a>, <Var p4_e>]
OutVars:[<Var p5_a>, <Var p5_e>]
A: 1*p1_a <= 2.0, -1*p1_a <= 0.0, 1*p1_e <= 0.0, -1*p1_e <= 0.0, 1*p2_a <= 3.75, -1*p2_a <= 0.0, 1*p2_e <= 0.0, -1*p2_e <= 0.0, 1*p4_a <= 0.0625, -1*p4_a <= 0.0, 1*p4_e <= 0.0, -1*p4_e <= 0.0
G: 1*p5_e <= 0.10000000000000009


The benefits of using contract is that we can focus on local information of the contract, and thus reduce the problem size.
As a result, we can perform exhaustive search on finding the wordlength that satisfy the goal.

In [24]:
def create_example1_by_p3(p3_n, p3_p):
    p1 = PortWordLength(n=5, p=2, name = "p1")
    p2 = PortWordLength(n=5, p=3, name = "p2")
    p3 = PortWordLength(n=p3_n, p=p3_p, name = "p3")
    c1 = form_contract_add(in_port1=p1, in_port2=p2, out_port=p3)

    p4 = PortWordLength(n=7, p=3, name = "p4")
    p5 = PortWordLength(n=6, p=3, name = "p5")
    c2 = form_contract_add(in_port1=p3, in_port2=p4, out_port=p5)

    p1.set_value("10000")
    p2.set_value("01111")
    p4.set_value("0000000")

    contract1 = read_contract(c1)
    contract2 = read_contract(c2)
    #print("Contract 1:\n" + str(contract1))
    #print("Contract 2:\n" + str(contract2))
    return contract1, contract2, p1, p2, p3, p4, p5

def compose_all(contract1, contract2, p1, p2, p3, p4, p5):
    c_p1 = form_contract_input(in_port=p1)
    c_p2 = form_contract_input(in_port=p2)
    c_p4 = form_contract_input(in_port=p4)

    contract_p1 = read_contract(c_p1)
    contract_p2 = read_contract(c_p2)
    contract_p4 = read_contract(c_p4)
    
    try: 
        contract_sys = contract1.compose(contract2)
        contract_sys = contract_p1.compose(contract_sys)
        contract_sys = contract_p2.compose(contract_sys)
        contract_sys = contract_p4.compose(contract_sys)
        print("Contract Sys:\n" + str(contract_sys))
        return contract_sys
    except ValueError as e:
        print("Composition Error")
        print(e)
        raise ValueError("Composition Fails")

#contract_sys = test_example_1()

def exhaustive_search():
    for n in range(5, 10):
        #print(n)
        p = 3 # keep the same number of bits (2) before binary point
        contract1, contract2, p1, p2, p3, p4, p5 = create_example1_by_p3(p3_n=n, p3_p=p)
        try:
            contract_sys = contract1.compose(contract2)
            #contract_sys = compose_all(contract1, contract2, p1, p2, p3, p4, p5)
            #print(str(contract_sys))
        except ValueError as e:
            continue

        if contract_sys.refines(quotient_ret):
            return n
n = exhaustive_search()
print(f"Wordlength for P3 to satisfy the spec is n = {n}")

    

Wordlength for P3 to satisfy the spec is n = 6


#### Example 2: Filter Design

We apply the contract to a simple filter that computes the moving average of the signal

![Example2](./figures/digital_filter_flow.png)

$y[n] = 0.2 \times x[n-2] + 0.6 \times x[n-1] + 0.2 \times x[n] $

And the following function perform the filter on the signal

In [25]:

def example2_filter(x):
    v = np.array([0.2, 0.6, 0.2])
    return np.convolve(x, v)

print(example2_filter([1.25, 2.25, 3.7, 6.5, 1.5]))


[0.25 1.2  2.34 3.97 4.94 2.2  0.3 ]


We can use contract to evaluate the abstracted performance of a datapath for the filter.

In the following, we examine the design of using 6 bits for all signal in a transpose form digital filter design.

In [26]:
in1 = PortWordLength(n=6, p=0, e=0, name="in1")
in2 = PortWordLength(n=6, p=0, e=0, name="in2")
in3 = PortWordLength(n=6, p=0, e=0, name="in3")
const1 = PortWordLength(n=6, p=0, name="const1")
const2 = PortWordLength(n=6, p=0, name="const2")
const3 = PortWordLength(n=6, p=0, name="const3")
mult_out1 = PortWordLength(n=6, p=0, name="mult_out1")
mult_out2 = PortWordLength(n=6, p=0, name="mult_out2")
mult_out3 = PortWordLength(n=6, p=0, name="mult_out3")
add_out1 = PortWordLength(n=6, p=0, name="add_out1")
add_out2 = PortWordLength(n=6, p=0, name="add_out2")

const1.set_value(float_to_bin(0.2, const1))
const2.set_value(float_to_bin(0.6, const2))
const3.set_value(float_to_bin(0.2, const3))
const1.set_e(0.2 - const1.value_num())
const2.set_e(0.6 - const2.value_num())
const3.set_e(0.2 - const3.value_num())

print(f"truncated coefficient: 0.2 to {const1.value_num()}")
print(f"truncated coefficient: 0.6 to {const2.value_num()}")
print(f"truncated coefficient: 0.2 to {const3.value_num()}")

c1 = form_contract_mult_const(in_port1=in1, in_port_const=const1, out_port=mult_out1)
c2 = form_contract_mult_const(in_port1=in2, in_port_const=const2, out_port=mult_out2)
c3 = form_contract_mult_const(in_port1=in3, in_port_const=const3, out_port=mult_out3)

ci1 = form_contract_input(in_port=in1)
ci2 = form_contract_input(in_port=in2)
ci3 = form_contract_input(in_port=in3)

contract1 = read_contract(c1)
contract2 = read_contract(c2)
contract3 = read_contract(c3)
contract_i1 = read_contract(ci1)
contract_i2 = read_contract(ci2)
contract_i3 = read_contract(ci3)
c4 = form_contract_add(in_port1 = mult_out1, in_port2=mult_out2, out_port=add_out1)
c5 = form_contract_add(in_port1 = add_out1, in_port2=mult_out3, out_port=add_out2)
contract4 = read_contract(c4)
contract5 = read_contract(c5)

contract_system = contract_i1.compose(contract1)
contract_system = contract_system.compose(contract_i2)
contract_system = contract_system.compose(contract2)
contract_system = contract_system.compose(contract_i3)
contract_system = contract_system.compose(contract3)
contract_system = contract_system.compose(contract4)
contract_system = contract_system.compose(contract5)
print(str(contract_system))


truncated coefficient: 0.2 to 0.1875
truncated coefficient: 0.6 to 0.59375
truncated coefficient: 0.2 to 0.1875
Port: , (n, p) = (12, 0), e = 0, a = 0.999755859375
Port: , (n, p) = (12, 0), e = 0, a = 0.999755859375
Port: , (n, p) = (12, 0), e = 0, a = 0.999755859375
InVars: []
OutVars:[<Var add_out2_a>, <Var add_out2_e>]
A: true
G: 1*add_out2_e <= 0.0769042968750000, 1*add_out2_a <= 0.953613281250000


The result shows that we can get an upper bound on the error of the output `add_out_e` is bounded by about 0.07690.
And this is the extreme case when the input is the following case:

Note that this is always an upper bound to the actual errors as we abstract the calculation and considers the worse case truncation error for each operation.

If the obtained error is smaller than the specification, we can rest assure that the design already satisfy the goal and we can either submit the design or change some wordlength and see if we can use smaller wordlength to achieve the same specification.

If the obtained error is larger than the specification, the level of abstraction is not sufficient to prove the correctness of the design.
Refinement of the error model or extensive verfication is therefore needed to verify the design.
 

To see the actual errors for this specific case, we can enumerate all input sequence.


In [27]:
def enumerate_error():
    diff_max = 0
    a_max = 0
    i1_max=0
    i2_max=0
    i3_max=0
    for i1 in range(0, 2**6):
        i1_float = i1/2**6
        in1_a = float_to_bin(i1_float, in1)
        in1.set_value(in1_a)
        for i2 in range(0, 2**6):
            i2_float = i2/2**6
            in2_a = float_to_bin(i2_float, in2)
            in2.set_value(in2_a)
            for i3 in range(0, 2**6):
                i3_float = i3/2**6
                in3_a = float_to_bin(i3_float, in3)
                in3.set_value(in3_a)

                port_mult(in1, const1, mult_out1)
                port_mult(in2, const2, mult_out2)
                port_mult(in3, const3, mult_out3)
                port_add(mult_out1, mult_out2, add_out1)
                port_add(add_out1, mult_out3, add_out2)
                actual_result = add_out2.value_num()
                ideal_result = i1_float * 0.2 + i2_float * 0.6 + i3_float * 0.2
                diff = ideal_result - actual_result

                if diff > diff_max:
                    diff_max = diff
                    i1_max = i1_float
                    i2_max = i2_float
                    i3_max = i3_float
                if actual_result > a_max:
                    a_max = actual_result
    return diff_max, i1_max, i2_max, i3_max, a_max

diff_max, i1_max, i2_max, i3_max, a_max = enumerate_error()
print(diff_max, i1_max, i2_max, i3_max, a_max)

0.06874999999999998 0.828125 0.578125 0.828125 0.921875


This verifies that our abstracted error bound is indeed an upper bound for all possible input values.